In [72]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US')

!pwd

/Users/mattbaldree/Google Drive/smu/quantifying/QTW_14/QTW_14


In [63]:
# flags
PURGE_DATA = True

plan of attack

- ~~download files in parallel~~
- view a file
- read files in parallel to csv format
- convert to parquet format
- read into df_p
- work with df

In [10]:
# start Dask distributed client and print out stats
from dask.distributed import Client
c = Client()
c

Client Scheduler: tcp://127.0.0.1:54761 Dashboard: http://127.0.0.1:54762,Cluster Workers: 4 Cores: 4 Memory: 10.31 GB


In [84]:
if PURGE_DATA:
    # delete /data directory
    import os
    from shutil import rmtree

    path = 'data'
    if os.path.exists(path):
        rmtree(path)
    
    # make /data if it doesn't exist
    path = 'data'
    if not os.path.exists(path):
        os.mkdir(path)

In [85]:
DOWNLOAD_ONE_FILE_ONLY = False

if PURGE_DATA:
    import urllib.request
    import shutil
    import bz2
    
    def download_file(baseurl, yr):
        file_name = ''

        url_of_data_file = baseurl%(yr)
        file_name = 'data/%d.csv'%(yr)
        size = 0

        print('downloading', url_of_data_file, 'to', file_name)
        decompressor = bz2.BZ2Decompressor()

        # download file and decompress it
        with urllib.request.urlopen(url_of_data_file) as response, open(file_name, 'wb') as out_file:
                data = decompressor.decompress(response.read())
                out_file.write(data)
                size = len(data)
                print('file size (MB)', locale.format('%.1f', size/1000000, grouping=True))

        return(file_name, size)
    
    def print_files(files):
        totalSize = 0        
        for f in files:
            size = f[1]/1000000
            totalSize += size
            print('downloaded file:', f[0], ', of size (MB):', 
                  locale.format('%.1f', size, grouping=True))
            
        print('Number of files downloaded:', len(files), 'for a total size (MB):', 
              locale.format('%.1f', totalSize, grouping=True))


    if DOWNLOAD_ONE_FILE_ONLY:
        # testing
        download_file('http://stat-computing.org/dataexpo/2009/%d.csv.bz2', 1987)
    else:    
        # download airline data from 1987 to 2009
        yrs = range(1987, 2009)
        baseurl = 'http://stat-computing.org/dataexpo/2009/%d.csv.bz2'

        from dask import delayed
        download_file = delayed(download_file)

        files = [download_file(baseurl, yr) for yr in yrs]
        files = delayed(files)

        %time files = files.compute()   
      
        print_files(files)

CPU times: user 36.4 s, sys: 10.5 s, total: 46.9 s
Wall time: 9min 11s
downloaded file: data/1987.csv , of size (MB): 127.2
downloaded file: data/1988.csv , of size (MB): 501.0
downloaded file: data/1989.csv , of size (MB): 486.5
downloaded file: data/1990.csv , of size (MB): 509.2
downloaded file: data/1991.csv , of size (MB): 491.2
downloaded file: data/1992.csv , of size (MB): 492.3
downloaded file: data/1993.csv , of size (MB): 490.8
downloaded file: data/1994.csv , of size (MB): 501.6
downloaded file: data/1995.csv , of size (MB): 530.8
downloaded file: data/1996.csv , of size (MB): 533.9
downloaded file: data/1997.csv , of size (MB): 540.3
downloaded file: data/1998.csv , of size (MB): 538.4
downloaded file: data/1999.csv , of size (MB): 552.9
downloaded file: data/2000.csv , of size (MB): 570.2
downloaded file: data/2001.csv , of size (MB): 600.4
downloaded file: data/2002.csv , of size (MB): 530.5
downloaded file: data/2003.csv , of size (MB): 626.7
downloaded file: data/2004.c

**Click on Dashboard URL printed out above to see the distributed work.**